# read in, clean up, data
inflation, EquityReturns, Unemployment,
consumer sentiment, home buyers index
get into linear form for dataframe, database

In [1]:
#dependencies
import sqlite3
import pandas as pd
import datetime
from datetime import timedelta

In [2]:
rawdatapath='EconomicDataRaw/'

### utility functions

In [3]:
# pass in dataframe with a date_time object as index
# return same dataframe, but index converted into YYYYMM as a number

def dt_to_yyyymm(df):

    years=df.index.year
    months=df.index.month
    new_index=years*100 + months
    df.index=new_index
    
    return df

In [4]:
## take a list of monthly prices (e.g. stock market, cpi), convert to yields. 
## will set initial yield to 0

def prices_to_yields(prices):
    numerator=prices[1:]  
    denominator=prices[:-1]
    yieldlist=[12*((n/d)-1) for n,d in zip(numerator,denominator)]
    yieldlist.insert(0,0)   # add a zero as first element to make string length correct
    
    return yieldlist


# read in CPI, convert to inflation, setup monthly dataframe¶


In [5]:
CPI_df=pd.read_excel(rawdatapath+'CPI_bls.xlsx',skiprows=10,header=1)

# create monthly dates to use in dataframe
start=datetime.datetime(2000,1,1)
end=datetime.datetime(2021,1,31)
months=pd.date_range(start,end,freq="BM")

#create dateframe df_econ, turn cpi from year/month table into monthly list
df_econ=pd.DataFrame({'Date':months})      

CPI=[]
for date in df_econ['Date']:
    cpival=CPI_df.loc[CPI_df["Year"]==date.year].iloc[0,[date.month]].values[0]  # values[0] turns 1 element array to a mumber
    CPI.append(cpival)
inflation=prices_to_yields(CPI)
df_econ['inflation']=inflation
df_econ.set_index('Date',inplace=True)
df_econ=dt_to_yyyymm(df_econ)
df_econ.head(7)

,inflation
Date,
200001,0.000000
200002,0.068802
200003,0.096717
200004,0.009360
200005,0.009353
200006,0.067757
200007,0.023233


# equity returns (S&P) 

In [6]:
SandP_df=pd.read_csv(rawdatapath+'SandP_Yahoo.csv')
# only column I want to keep is Close; drop everything else
SandP_df.drop(columns=['Open','High','Low','Adj Close','Volume'],inplace=True) #just keep close price

# turn "Date" column into date-time object, then convert to yyyymm
dto=[]                      # list to hold date time object
for date in SandP_df['Date']:
    dto.append(datetime.datetime.strptime(date,'%Y-%m-%d')) # convert string to date_time object
SandP_df['DTdate']=dto
SandP_df.set_index('DTdate',inplace=True)
SandP_df.drop(columns=['Date'],inplace=True)
SandP_df=dt_to_yyyymm(SandP_df)
yields=prices_to_yields(SandP_df['Close'].values)
SandP_df['S&Pyield']=yields
SandP_df.drop(columns=['Close'],inplace=True)
SandP_df.head()

#merge into df_econ
#df_econ=pd.concat([df_econ,SandP_df],axis=1,join='inner')
#df_econ

,S&Pyield
DTdate,
198501,0.000000
198502,0.103545
198503,-0.034440
198504,-0.055131
198505,0.648613


In [7]:
df_econ=pd.concat([df_econ,SandP_df],axis=1,join='inner')
df_econ

,inflation,S&Pyield
200001,0.000000,-0.610843
200002,0.068802,-0.241297
200003,0.096717,1.160638
200004,0.009360,-0.369549
200005,0.009353,-0.262981
...,...,...
202009,0.016722,-0.470736
202010,0.004987,-0.331989
202011,-0.007338,1.290548
202012,0.011268,0.445458


### unemployment

In [8]:
table_df=pd.read_excel(rawdatapath+'HistoricalUnemployment_bls.xlsx',skiprows=10,header=1)

# create monthly dates to use in dataframe
start=datetime.datetime(2000,1,1)
end=datetime.datetime(2021,1,31)
months=pd.date_range(start,end,freq="BM")

#create dateframe df_econ, turn cpi from year/month table into monthly list
df=pd.DataFrame({'Date':months})      

unemp_rate=[]
for date in df['Date']:
    unempval=table_df.loc[table_df["Year"]==date.year].iloc[0,[date.month]].values[0]  # values[0] turns 1 element array to a mumber
    unemp_rate.append(unempval)
df['unemployment']=unemp_rate
df.set_index('Date',inplace=True)
df=dt_to_yyyymm(df)
df.head(7)

,unemployment
Date,
200001,4.0
200002,4.1
200003,4.0
200004,3.8
200005,4.0
200006,4.0
200007,4.0


In [9]:
df_econ=pd.concat([df_econ,df],axis=1,join='inner')
df_econ

,inflation,S&Pyield,unemployment
200001,0.000000,-0.610843,4.0
200002,0.068802,-0.241297,4.1
200003,0.096717,1.160638,4.0
200004,0.009360,-0.369549,3.8
200005,0.009353,-0.262981,4.0
...,...,...,...
202009,0.016722,-0.470736,7.8
202010,0.004987,-0.331989,6.9
202011,-0.007338,1.290548,6.7
202012,0.011268,0.445458,6.7


# Home Buyers Index
Change format so that there is a column for year, and a column for month.
Fill null data with prior month's info.
ALSO: We need dates to align by last day of month - Subtract by one day.

In [54]:
HomeBuyers_df=pd.read_excel(rawdatapath+'HomeBuyerIndex_fhfa.xls',header=1)
HomeBuyers_df

,Region,Year,Quarter,Not Seasonally-Adjusted\nPurchase-Only Index \n(1991Q1=100),Seasonally-Adjusted Purchase-Only Index \n(1991Q1=100),Not Seasonally-Adjusted\nPurchase-Only Index\n% Change Over \nPrevious Quarter,Seasonally-Adjusted Purchase-Only Index\n% Change Over \nPrevious Quarter,Not Seasonally-Adjusted\nPurchase-Only Index\n% Change Over \nPrevious 4 Quarters,Seasonally-Adjusted Purchase-Only Index\n% Change Over \nPrevious 4 Quarters
0,USA,1991,1,100.00,100.00,NaN,NaN,NaN,NaN
1,USA,1991,2,100.49,99.98,0.0049,-0.0002,NaN,NaN
2,USA,1991,3,100.79,100.18,0.0029,0.0019,NaN,NaN
3,USA,1991,4,101.44,101.05,0.0065,0.0087,NaN,NaN
4,USA,1992,1,102.25,102.27,0.0079,0.0120,0.0225,0.0227
...,...,...,...,...,...,...,...,...,...
115,USA,2019,4,275.72,275.45,0.0041,0.0129,0.0546,0.0536
116,USA,2020,1,280.63,281.18,0.0178,0.0208,0.0621,0.0615
117,USA,2020,2,287.22,284.22,0.0235,0.0108,0.0569,0.0576
118,USA,2020,3,296.66,293.92,0.0329,0.0341,0.0803,0.0808


In [55]:
HomeBuyers_df.isnull().sum()


Region                                                                               0
Year                                                                                 0
Quarter                                                                              0
Not Seasonally-Adjusted\nPurchase-Only Index \n(1991Q1=100)                          0
Seasonally-Adjusted Purchase-Only Index \n(1991Q1=100)                               0
Not Seasonally-Adjusted\nPurchase-Only Index\n% Change Over \nPrevious Quarter       1
Seasonally-Adjusted Purchase-Only Index\n% Change Over \nPrevious Quarter            1
Not Seasonally-Adjusted\nPurchase-Only Index\n% Change Over \nPrevious 4 Quarters    4
Seasonally-Adjusted Purchase-Only Index\n% Change Over \nPrevious 4 Quarters         4
dtype: int64

In [56]:
#Drop calculated columns
HomeBuyers_df.drop(HomeBuyers_df.columns[[3,4,5,6,7]], axis=1, inplace=True)
HomeBuyers_df

,Region,Year,Quarter,Seasonally-Adjusted Purchase-Only Index\n% Change Over \nPrevious 4 Quarters
0,USA,1991,1,NaN
1,USA,1991,2,NaN
2,USA,1991,3,NaN
3,USA,1991,4,NaN
4,USA,1992,1,0.0227
...,...,...,...,...
115,USA,2019,4,0.0536
116,USA,2020,1,0.0615
117,USA,2020,2,0.0576
118,USA,2020,3,0.0808


In [59]:
HomeBuyers_df.rename(columns={"Seasonally-Adjusted Purchase-Only Index\n% Change Over \nPrevious 4 Quarters":"homeindex"},inplace=True)

#HomeBuyers_df['quarter2'] = HomeBuyers_df[(HomeBuyers_df["Year"]) & (HomeBuyers_df["Quarter"])]
df2 = HomeBuyers_df.assign(ColumnA = HomeBuyers_df.Year.astype(str) + 'Q' + HomeBuyers_df.Quarter.astype(str))
df2

,Region,Year,Quarter,homeindex,ColumnA
0,USA,1991,1,NaN,1991Q1
1,USA,1991,2,NaN,1991Q2
2,USA,1991,3,NaN,1991Q3
3,USA,1991,4,NaN,1991Q4
4,USA,1992,1,0.0227,1992Q1
...,...,...,...,...,...
115,USA,2019,4,0.0536,2019Q4
116,USA,2020,1,0.0615,2020Q1
117,USA,2020,2,0.0576,2020Q2
118,USA,2020,3,0.0808,2020Q3


In [14]:
# Build df for each desired timepoint (monthly from 1991-2020)
Dates = []

start_date = pd.to_datetime('01-01-1991')
for j in pd.date_range(start_date, freq='M', periods=360):
  Dates.append(j)

In [15]:
MonthlyHB_df = pd.DataFrame(Dates)
MonthlyHB_df = MonthlyHB_df.set_axis(["date"], axis=1)

MonthlyHB_df

,date
0,1991-01-31
1,1991-02-28
2,1991-03-31
3,1991-04-30
4,1991-05-31
...,...
355,2020-08-31
356,2020-09-30
357,2020-10-31
358,2020-11-30


In [16]:
# Convert the date to match 'ColumnA' for a merge
MonthlyHB_df['ColumnA'] = pd.PeriodIndex(MonthlyHB_df.date, freq='Q').astype(str)
MonthlyHB_df

,date,ColumnA
0,1991-01-31,1991Q1
1,1991-02-28,1991Q1
2,1991-03-31,1991Q1
3,1991-04-30,1991Q2
4,1991-05-31,1991Q2
...,...,...
355,2020-08-31,2020Q3
356,2020-09-30,2020Q3
357,2020-10-31,2020Q4
358,2020-11-30,2020Q4


In [60]:
#Merge dataframes on ColumnA

left_join_df= pd.merge(MonthlyHB_df, df2, on='ColumnA', how='left')
left_join_df 

,date,ColumnA,Region,Year,Quarter,homeindex
0,1991-01-31,1991Q1,USA,1991,1,NaN
1,1991-02-28,1991Q1,USA,1991,1,NaN
2,1991-03-31,1991Q1,USA,1991,1,NaN
3,1991-04-30,1991Q2,USA,1991,2,NaN
4,1991-05-31,1991Q2,USA,1991,2,NaN
...,...,...,...,...,...,...
355,2020-08-31,2020Q3,USA,2020,3,0.0808
356,2020-09-30,2020Q3,USA,2020,3,0.0808
357,2020-10-31,2020Q4,USA,2020,4,0.1077
358,2020-11-30,2020Q4,USA,2020,4,0.1077


In [61]:
left_join_df.drop(columns=['ColumnA','Region','Year','Quarter'],inplace=True)
left_join_df

,date,homeindex
0,1991-01-31,NaN
1,1991-02-28,NaN
2,1991-03-31,NaN
3,1991-04-30,NaN
4,1991-05-31,NaN
...,...,...
355,2020-08-31,0.0808
356,2020-09-30,0.0808
357,2020-10-31,0.1077
358,2020-11-30,0.1077


In [63]:
homeindex_df= left_join_df
homeindex_df.set_index('date',inplace=True)

homeindex_df=dt_to_yyyymm(homeindex_df)
homeindex_df

,homeindex
date,
199101,NaN
199102,NaN
199103,NaN
199104,NaN
199105,NaN
...,...
202008,0.0808
202009,0.0808
202010,0.1077


In [66]:
df_econ=pd.concat([df_econ,homeindex_df],axis=1,join='inner')
df_econ

,inflation,S&Pyield,unemployment,UMCSENT,year,month,homeindex
200001,0.000000,-0.610843,4.0,111.3,2000,1,0.0646
200002,0.068802,-0.241297,4.1,107.1,2000,2,0.0646
200003,0.096717,1.160638,4.0,109.2,2000,3,0.0646
200004,0.009360,-0.369549,3.8,110.7,2000,4,0.0666
200005,0.009353,-0.262981,4.0,106.4,2000,5,0.0666
...,...,...,...,...,...,...,...
202008,0.037833,0.840776,8.4,80.4,2020,8,0.0808
202009,0.016722,-0.470736,7.8,81.8,2020,9,0.0808
202010,0.004987,-0.331989,6.9,76.9,2020,10,0.1077
202011,-0.007338,1.290548,6.7,80.7,2020,11,0.1077


# Consumer Sentiment
Change format so that there is a column for year, and a column for month.
Fill null data with prior month's info

In [67]:
ConsumerSentiment_df=pd.read_excel(rawdatapath+'UMCSENT_fred.xls',skiprows=9,header=1)
ConsumerSentiment_df

,observation_date,UMCSENT
0,1952-11-01,86.2
1,1952-12-01,NaN
2,1953-01-01,NaN
3,1953-02-01,90.7
4,1953-03-01,NaN
...,...,...
814,2020-09-01,80.4
815,2020-10-01,81.8
816,2020-11-01,76.9
817,2020-12-01,80.7


In [68]:
today = datetime.datetime.today() - timedelta(days=1)
print(today)

2021-03-05 15:58:20.546807


In [69]:
#Subtract one day from the date to get end of month data
ConsumerSentiment_df['End of Month'] = ConsumerSentiment_df.apply(lambda row: row.observation_date - timedelta(days=1), axis = 1) 
ConsumerSentiment_df

,observation_date,UMCSENT,End of Month
0,1952-11-01,86.2,1952-10-31
1,1952-12-01,NaN,1952-11-30
2,1953-01-01,NaN,1952-12-31
3,1953-02-01,90.7,1953-01-31
4,1953-03-01,NaN,1953-02-28
...,...,...,...
814,2020-09-01,80.4,2020-08-31
815,2020-10-01,81.8,2020-09-30
816,2020-11-01,76.9,2020-10-31
817,2020-12-01,80.7,2020-11-30


In [70]:
#Break date into year and month columns
ConsumerSentiment_df['year'] = pd.DatetimeIndex(ConsumerSentiment_df['End of Month']).year
ConsumerSentiment_df['month'] = pd.DatetimeIndex(ConsumerSentiment_df['End of Month']).month
ConsumerSentiment_df

,observation_date,UMCSENT,End of Month,year,month
0,1952-11-01,86.2,1952-10-31,1952,10
1,1952-12-01,NaN,1952-11-30,1952,11
2,1953-01-01,NaN,1952-12-31,1952,12
3,1953-02-01,90.7,1953-01-31,1953,1
4,1953-03-01,NaN,1953-02-28,1953,2
...,...,...,...,...,...
814,2020-09-01,80.4,2020-08-31,2020,8
815,2020-10-01,81.8,2020-09-30,2020,9
816,2020-11-01,76.9,2020-10-31,2020,10
817,2020-12-01,80.7,2020-11-30,2020,11


In [71]:
# Fix the NaN UMCSENT scores...
ConsumerSentiment_df.isnull().sum()

observation_date      0
UMCSENT             210
End of Month          0
year                  0
month                 0
dtype: int64

In [72]:
ConsumerSentiment_df.count()

observation_date    819
UMCSENT             609
End of Month        819
year                819
month               819
dtype: int64

In [73]:
#Option1: Drop null data
CS_NullsRemoved = ConsumerSentiment_df.dropna()
CS_NullsRemoved

,observation_date,UMCSENT,End of Month,year,month
0,1952-11-01,86.2,1952-10-31,1952,10
3,1953-02-01,90.7,1953-01-31,1953,1
9,1953-08-01,80.8,1953-07-31,1953,7
12,1953-11-01,80.7,1953-10-31,1953,10
15,1954-02-01,82.0,1954-01-31,1954,1
...,...,...,...,...,...
814,2020-09-01,80.4,2020-08-31,2020,8
815,2020-10-01,81.8,2020-09-30,2020,9
816,2020-11-01,76.9,2020-10-31,2020,10
817,2020-12-01,80.7,2020-11-30,2020,11


In [74]:
#Option2: Generate 210 results: Replace nulls with last non-null score
CS_LastScoreFill = ConsumerSentiment_df.fillna(method="ffill")
CS_LastScoreFill

,observation_date,UMCSENT,End of Month,year,month
0,1952-11-01,86.2,1952-10-31,1952,10
1,1952-12-01,86.2,1952-11-30,1952,11
2,1953-01-01,86.2,1952-12-31,1952,12
3,1953-02-01,90.7,1953-01-31,1953,1
4,1953-03-01,90.7,1953-02-28,1953,2
...,...,...,...,...,...
814,2020-09-01,80.4,2020-08-31,2020,8
815,2020-10-01,81.8,2020-09-30,2020,9
816,2020-11-01,76.9,2020-10-31,2020,10
817,2020-12-01,80.7,2020-11-30,2020,11


In [75]:
CS_LastScoreFill = CS_LastScoreFill.drop(['observation_date', 'End of Month'], axis=1)
CS_LastScoreFill

,UMCSENT,year,month
0,86.2,1952,10
1,86.2,1952,11
2,86.2,1952,12
3,90.7,1953,1
4,90.7,1953,2
...,...,...,...
814,80.4,2020,8
815,81.8,2020,9
816,76.9,2020,10
817,80.7,2020,11


In [76]:
dateyyyymm=100*CS_LastScoreFill['year']+CS_LastScoreFill['month']
CS_LastScoreFill.set_index(dateyyyymm,inplace=True)
CS_LastScoreFill

,UMCSENT,year,month
195210,86.2,1952,10
195211,86.2,1952,11
195212,86.2,1952,12
195301,90.7,1953,1
195302,90.7,1953,2
...,...,...,...
202008,80.4,2020,8
202009,81.8,2020,9
202010,76.9,2020,10
202011,80.7,2020,11


In [77]:
df_econ

,inflation,S&Pyield,unemployment,UMCSENT,year,month,homeindex
200001,0.000000,-0.610843,4.0,111.3,2000,1,0.0646
200002,0.068802,-0.241297,4.1,107.1,2000,2,0.0646
200003,0.096717,1.160638,4.0,109.2,2000,3,0.0646
200004,0.009360,-0.369549,3.8,110.7,2000,4,0.0666
200005,0.009353,-0.262981,4.0,106.4,2000,5,0.0666
...,...,...,...,...,...,...,...
202008,0.037833,0.840776,8.4,80.4,2020,8,0.0808
202009,0.016722,-0.470736,7.8,81.8,2020,9,0.0808
202010,0.004987,-0.331989,6.9,76.9,2020,10,0.1077
202011,-0.007338,1.290548,6.7,80.7,2020,11,0.1077


In [78]:
df_econ=pd.concat([df_econ,CS_LastScoreFill],axis=1,join='inner')
df_econ

,inflation,S&Pyield,unemployment,UMCSENT,year,month,homeindex,UMCSENT,year,month
200001,0.000000,-0.610843,4.0,111.3,2000,1,0.0646,111.3,2000,1
200002,0.068802,-0.241297,4.1,107.1,2000,2,0.0646,107.1,2000,2
200003,0.096717,1.160638,4.0,109.2,2000,3,0.0646,109.2,2000,3
200004,0.009360,-0.369549,3.8,110.7,2000,4,0.0666,110.7,2000,4
200005,0.009353,-0.262981,4.0,106.4,2000,5,0.0666,106.4,2000,5
...,...,...,...,...,...,...,...,...,...,...
202008,0.037833,0.840776,8.4,80.4,2020,8,0.0808,80.4,2020,8
202009,0.016722,-0.470736,7.8,81.8,2020,9,0.0808,81.8,2020,9
202010,0.004987,-0.331989,6.9,76.9,2020,10,0.1077,76.9,2020,10
202011,-0.007338,1.290548,6.7,80.7,2020,11,0.1077,80.7,2020,11


    ### cleanup

In [80]:
df_econ.drop(columns=['month','year'],inplace=True)
df_econ

,inflation,S&Pyield,unemployment,UMCSENT,homeindex,UMCSENT
200001,0.000000,-0.610843,4.0,111.3,0.0646,111.3
200002,0.068802,-0.241297,4.1,107.1,0.0646,107.1
200003,0.096717,1.160638,4.0,109.2,0.0646,109.2
200004,0.009360,-0.369549,3.8,110.7,0.0666,110.7
200005,0.009353,-0.262981,4.0,106.4,0.0666,106.4
...,...,...,...,...,...,...
202008,0.037833,0.840776,8.4,80.4,0.0808,80.4
202009,0.016722,-0.470736,7.8,81.8,0.0808,81.8
202010,0.004987,-0.331989,6.9,76.9,0.1077,76.9
202011,-0.007338,1.290548,6.7,80.7,0.1077,80.7


In [81]:
df_econ.to_csv('df_econ.csv')